In [1]:
import pyspark
from pyspark.sql import SparkSession
import findspark
findspark.init()
findspark.find()
spark=SparkSession.builder.appName("jupyter").getOrCreate()


In [2]:
#constants
filepath=r"C:\Users\abhishek\Documents\excel_work\stratascratch.xls"

In [16]:
rdd=spark.sparkContext.parallelize(range(23,100))#.count()
rdd.count()

77

In [89]:
import pandas as pd
sheet="flag_review"
df2=pd.read_excel(filepath,sheet_name=sheet)
df2.head()

,flag_id,reviewed_by_yt,reviewed_date,reviewed_outcome
0,0cazx3,False,NaT,NaN
1,1cn76u,True,2022-03-15,REMOVED
2,1i43zk,True,2022-03-15,REMOVED
3,1n0vef,True,2022-03-15,REMOVED
4,1sv6ib,True,2022-03-15,APPROVED


In [90]:
#from pyspark.pandas import read_excel
df=spark.createDataFrame(df2)
df.show()

+-------+--------------+-------------------+----------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|
+-------+--------------+-------------------+----------------+
| 0cazx3|         false|               null|             NaN|
| 1cn76u|          true|2022-03-15 00:00:00|         REMOVED|
| 1i43zk|          true|2022-03-15 00:00:00|         REMOVED|
| 1n0vef|          true|2022-03-15 00:00:00|         REMOVED|
| 1sv6ib|          true|2022-03-15 00:00:00|        APPROVED|
| 20xekb|          true|2022-03-17 00:00:00|         REMOVED|
| 4cvwuv|          true|2022-03-15 00:00:00|        APPROVED|
| 4l1tk7|         false|               null|             NaN|
| 4sd6dv|          true|2022-03-14 00:00:00|         REMOVED|
| 6jjkvn|          true|2022-03-16 00:00:00|        APPROVED|
| 7ks264|          true|2022-03-15 00:00:00|        APPROVED|
| 8946nx|         false|               null|             NaN|
| 8wwg0l|         false|               null|             NaN|
| arydfd

In [20]:
df.groupby("reviewed_by_yt").count().orderBy('count',descending=False).limit(2).select("reviewed_by_yt").rdd.flatMap(lambda x:x).collect()

[False, True]

In [29]:

df.dropna(how="any",subset=["reviewed_outcome"]).show()#,col="reviewed_outcome")

+-------+--------------+-------------------+----------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|
+-------+--------------+-------------------+----------------+
| 0cazx3|         false|               null|             NaN|
| 1cn76u|          true|2022-03-15 00:00:00|         REMOVED|
| 1i43zk|          true|2022-03-15 00:00:00|         REMOVED|
| 1n0vef|          true|2022-03-15 00:00:00|         REMOVED|
| 1sv6ib|          true|2022-03-15 00:00:00|        APPROVED|
| 20xekb|          true|2022-03-17 00:00:00|         REMOVED|
| 4cvwuv|          true|2022-03-15 00:00:00|        APPROVED|
| 4l1tk7|         false|               null|             NaN|
| 4sd6dv|          true|2022-03-14 00:00:00|         REMOVED|
| 6jjkvn|          true|2022-03-16 00:00:00|        APPROVED|
| 7ks264|          true|2022-03-15 00:00:00|        APPROVED|
| 8946nx|         false|               null|             NaN|
| 8wwg0l|         false|               null|             NaN|
| arydfd

In [35]:
df.filter(df.reviewed_outcome!='NaN').show()

+-------+--------------+-------------------+----------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|
+-------+--------------+-------------------+----------------+
| 1cn76u|          true|2022-03-15 00:00:00|         REMOVED|
| 1i43zk|          true|2022-03-15 00:00:00|         REMOVED|
| 1n0vef|          true|2022-03-15 00:00:00|         REMOVED|
| 1sv6ib|          true|2022-03-15 00:00:00|        APPROVED|
| 20xekb|          true|2022-03-17 00:00:00|         REMOVED|
| 4cvwuv|          true|2022-03-15 00:00:00|        APPROVED|
| 4sd6dv|          true|2022-03-14 00:00:00|         REMOVED|
| 6jjkvn|          true|2022-03-16 00:00:00|        APPROVED|
| 7ks264|          true|2022-03-15 00:00:00|        APPROVED|
| arydfd|          true|2022-03-15 00:00:00|        APPROVED|
| bl40qw|          true|2022-03-16 00:00:00|         REMOVED|
| ehn1pt|          true|2022-03-18 00:00:00|        APPROVED|
| i2l3oo|          true|2022-03-17 00:00:00|         REMOVED|
| kc41jd

In [44]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType

@udf(returnType=StringType()) 
def capitaliseUDF(row):
    #if isinstance(row,str):
    if row:
        row=1
    else:
        row=0
    return row
#capitaliseUDF=udf(lambda z: capitalise(z))
df.select("reviewed_by_yt",capitaliseUDF("reviewed_by_yt").alias("reviewed_by_yt_upper")).show()

+--------------+--------------------+
|reviewed_by_yt|reviewed_by_yt_upper|
+--------------+--------------------+
|         false|                   0|
|          true|                   1|
|          true|                   1|
|          true|                   1|
|          true|                   1|
|          true|                   1|
|          true|                   1|
|         false|                   0|
|          true|                   1|
|          true|                   1|
|          true|                   1|
|         false|                   0|
|         false|                   0|
|          true|                   1|
|          true|                   1|
|          true|                   1|
|         false|                   0|
|          true|                   1|
|         false|                   0|
|         false|                   0|
+--------------+--------------------+
only showing top 20 rows



In [45]:
df.withColumn("reviewed_by_yt_upper",capitaliseUDF("reviewed_by_yt")).show()

+-------+--------------+-------------------+----------------+--------------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|reviewed_by_yt_upper|
+-------+--------------+-------------------+----------------+--------------------+
| 0cazx3|         false|               null|             NaN|                   0|
| 1cn76u|          true|2022-03-15 00:00:00|         REMOVED|                   1|
| 1i43zk|          true|2022-03-15 00:00:00|         REMOVED|                   1|
| 1n0vef|          true|2022-03-15 00:00:00|         REMOVED|                   1|
| 1sv6ib|          true|2022-03-15 00:00:00|        APPROVED|                   1|
| 20xekb|          true|2022-03-17 00:00:00|         REMOVED|                   1|
| 4cvwuv|          true|2022-03-15 00:00:00|        APPROVED|                   1|
| 4l1tk7|         false|               null|             NaN|                   0|
| 4sd6dv|          true|2022-03-14 00:00:00|         REMOVED|                   1|
| 6j

In [46]:
@udf
def count_characters(row):
    if row:
        return len(row)

df.withColumn("flag_id_wordcount",count_characters("flag_id")).show()

+-------+--------------+-------------------+----------------+-----------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|flag_id_wordcount|
+-------+--------------+-------------------+----------------+-----------------+
| 0cazx3|         false|               null|             NaN|                6|
| 1cn76u|          true|2022-03-15 00:00:00|         REMOVED|                6|
| 1i43zk|          true|2022-03-15 00:00:00|         REMOVED|                6|
| 1n0vef|          true|2022-03-15 00:00:00|         REMOVED|                6|
| 1sv6ib|          true|2022-03-15 00:00:00|        APPROVED|                6|
| 20xekb|          true|2022-03-17 00:00:00|         REMOVED|                6|
| 4cvwuv|          true|2022-03-15 00:00:00|        APPROVED|                6|
| 4l1tk7|         false|               null|             NaN|                6|
| 4sd6dv|          true|2022-03-14 00:00:00|         REMOVED|                6|
| 6jjkvn|          true|2022-03-16 00:00

In [47]:
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df_difference = spark.createDataFrame(data, ["name", "age" , "salary"])
df_difference.show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+



In [61]:
from pyspark.sql.functions import col, lag, desc,asc
from pyspark.sql.window import Window

window=Window().orderBy(asc(col("salary")))#.descending
#.partitionBy("age").orderBy("name")

df_difference1=df_difference.withColumn("lag_salary",lag(df_difference.salary).over(window))
df_difference1.show()

+-------+---+------+----------+----------------+
|   name|age|salary|lag_salary|differece_salary|
+-------+---+------+----------+----------------+
|  James| 34| 55000|      null|            null|
| Robert| 37| 60000|     55000|            5000|
|    Jen| 32| 65000|     60000|            5000|
|Michael| 30| 70000|     65000|            5000|
|  Maria| 29| 80000|     70000|           10000|
+-------+---+------+----------+----------------+



In [60]:
df_difference=df_difference1.withColumn("differece_salary",df_difference1.salary-df_difference1.lag_salary)
df_difference.show()

+-------+---+------+----------+----------------+
|   name|age|salary|lag_salary|differece_salary|
+-------+---+------+----------+----------------+
|  James| 34| 55000|      null|            null|
| Robert| 37| 60000|     55000|            5000|
|    Jen| 32| 65000|     60000|            5000|
|Michael| 30| 70000|     65000|            5000|
|  Maria| 29| 80000|     70000|           10000|
+-------+---+------+----------+----------------+



In [83]:
from pyspark.sql.functions import expr, explode, sequence, rand

# Start date and end date (start + 10 weekends)
start_date = '2000-01-01'
end_date = '2000-03-04' # Calculated manually: 10 weekends (Saturdays) from start date

# Create a DataFrame with one row containing a sequence from start_date to end_date with a 1 day step
df = spark.range(1).select(
explode(
sequence(
expr(f"date '{start_date}'"),
expr(f"date '{end_date}'"),
expr("interval 1 day")
)
).alias("date")
)

# Filter out the weekdays (retain weekends)
df = df.filter(expr("dayofweek(date) = 7")) # 7 corresponds to Saturday in Spark

# Add the random numbers column
#df = df.withColumn("random_numbers", rand()*10)
df = df.withColumn("random_numbers", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()


+----------+--------------+
|      date|random_numbers|
+----------+--------------+
|2000-01-01|             8|
|2000-01-08|             3|
|2000-01-15|             8|
|2000-01-22|             5|
|2000-01-29|             4|
|2000-02-05|             6|
|2000-02-12|             8|
|2000-02-19|             1|
|2000-02-26|             9|
|2000-03-04|             3|
+----------+--------------+



In [88]:
df=spark.range(1).select(
    explode(
        sequence(
            expr(f"date '{start_date}'"),
            expr(f"date '{end_date}'"),
            expr("interval 1 day"))))
df.show()

+----------+
|       col|
+----------+
|2000-01-01|
|2000-01-02|
|2000-01-03|
|2000-01-04|
|2000-01-05|
|2000-01-06|
|2000-01-07|
|2000-01-08|
|2000-01-09|
|2000-01-10|
|2000-01-11|
|2000-01-12|
|2000-01-13|
|2000-01-14|
|2000-01-15|
|2000-01-16|
|2000-01-17|
|2000-01-18|
|2000-01-19|
|2000-01-20|
+----------+
only showing top 20 rows



In [91]:
df.show()

+-------+--------------+-------------------+----------------+
|flag_id|reviewed_by_yt|      reviewed_date|reviewed_outcome|
+-------+--------------+-------------------+----------------+
| 0cazx3|         false|               null|             NaN|
| 1cn76u|          true|2022-03-15 00:00:00|         REMOVED|
| 1i43zk|          true|2022-03-15 00:00:00|         REMOVED|
| 1n0vef|          true|2022-03-15 00:00:00|         REMOVED|
| 1sv6ib|          true|2022-03-15 00:00:00|        APPROVED|
| 20xekb|          true|2022-03-17 00:00:00|         REMOVED|
| 4cvwuv|          true|2022-03-15 00:00:00|        APPROVED|
| 4l1tk7|         false|               null|             NaN|
| 4sd6dv|          true|2022-03-14 00:00:00|         REMOVED|
| 6jjkvn|          true|2022-03-16 00:00:00|        APPROVED|
| 7ks264|          true|2022-03-15 00:00:00|        APPROVED|
| 8946nx|         false|               null|             NaN|
| 8wwg0l|         false|               null|             NaN|
| arydfd

In [92]:
import pyspark.sql.functions as F


DataFrame[]

In [104]:
df.select(
    F.when(
        F.isnan(df.reviewed_outcome)
        |df.reviewed_outcome.contains("REMOVED")
        |df.reviewed_outcome.contains("None")
        |df.reviewed_outcome.isNull()
        , 0)
    .otherwise(1).alias("null_count")).show()


+----------+
|null_count|
+----------+
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         1|
|         0|
|         0|
|         1|
|         1|
|         0|
|         0|
|         1|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [108]:
filepath=r"C:\Users\abhishek\Documents\coding\ds-algo\interview_test\publicis_sapient.txt"
df = spark.read.option("header", "true") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv(filepath)
df.show()

+---------------------------------+
|Give a timeseries data and userid|
+---------------------------------+
|             . find the sessio...|
|             . the user is log...|
|             Find the user act...|
|             Find how much tim...|
|             timestamp        ...|
|             2023-01-01 1:05:0...|
|             2023-01-01 2:05:0...|
|             2023-01-12 1:05:0...|
|             2023-01-01 1:05:0...|
|             2023-01-12 1:35:0...|
+---------------------------------+



In [180]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("userid", StringType(), True)
])
schema="""
timestamp string,
userid string
"""

In [182]:
#schema=
df = spark.read.csv(filepath, schema=schema, sep="\t")
df.show()

+--------------------+------+
|           timestamp|userid|
+--------------------+------+
|2023-01-01 1:05:0...|  null|
|2023-01-01 2:05:0...|  null|
|2023-01-12 1:05:0...|  null|
|2023-01-01 1:05:0...|  null|
|2023-01-12 1:35:0...|  null|
+--------------------+------+



In [187]:
df.select(F.split(F.col("timestamp"),"  ")).show()#"timestamp")

+------------------------+
|split(timestamp,   , -1)|
+------------------------+
|    [2023-01-01 1:05:...|
|    [2023-01-01 2:05:...|
|    [2023-01-12 1:05:...|
|    [2023-01-01 1:05:...|
|    [2023-01-12 1:35:...|
+------------------------+



In [234]:
sc=spark.sparkContext.textFile(filepath)

In [235]:
val=sc.map(lambda row: row.split(" ")).collect()
val

[['2023-01-01', '1:05:00', '', '', '', '', '', 'u1'],
 ['2023-01-01', '1:25:00', '', '', '', '', '', 'u1'],
 ['2023-01-02', '2:05:00', '', '', '', '', '', 'u1'],
 ['2023-01-02', '2:35:00', '', '', '', '', '', 'u1'],
 ['2023-01-02', '3:05:00', '', '', '', '', '', 'u1'],
 ['2023-01-12', '1:05:00', '', '', '', '', '', 'u2'],
 ['2023-01-12', '1:05:00', '', '', '', '', '', 'u2'],
 ['2023-01-12', '1:05:00', '', '', '', '', '', 'u2'],
 ['2023-01-01', '1:05:00', '', '', '', '', '', 'u1'],
 ['2023-01-12', '1:35:00', '', '', '', '', '', 'u2']]

In [236]:
df_date=spark.createDataFrame(val)
df_date.show()

+----------+-------+---+---+---+---+---+---+
|        _1|     _2| _3| _4| _5| _6| _7| _8|
+----------+-------+---+---+---+---+---+---+
|2023-01-01|1:05:00|   |   |   |   |   | u1|
|2023-01-01|1:25:00|   |   |   |   |   | u1|
|2023-01-02|2:05:00|   |   |   |   |   | u1|
|2023-01-02|2:35:00|   |   |   |   |   | u1|
|2023-01-02|3:05:00|   |   |   |   |   | u1|
|2023-01-12|1:05:00|   |   |   |   |   | u2|
|2023-01-12|1:05:00|   |   |   |   |   | u2|
|2023-01-12|1:05:00|   |   |   |   |   | u2|
|2023-01-01|1:05:00|   |   |   |   |   | u1|
|2023-01-12|1:35:00|   |   |   |   |   | u2|
+----------+-------+---+---+---+---+---+---+



In [237]:
df_date_formatted=df_date.select("_1","_2","_8").withColumnRenamed("_1","date").withColumnRenamed("_2","time").withColumnRenamed("_8","userid")
df_date_formatted.show()

+----------+-------+------+
|      date|   time|userid|
+----------+-------+------+
|2023-01-01|1:05:00|    u1|
|2023-01-01|1:25:00|    u1|
|2023-01-02|2:05:00|    u1|
|2023-01-02|2:35:00|    u1|
|2023-01-02|3:05:00|    u1|
|2023-01-12|1:05:00|    u2|
|2023-01-12|1:05:00|    u2|
|2023-01-12|1:05:00|    u2|
|2023-01-01|1:05:00|    u1|
|2023-01-12|1:35:00|    u2|
+----------+-------+------+



In [238]:
df_date_formatted=df_date_formatted.withColumn("datetime_",F.concat_ws(" ",df_date_formatted.date,df_date_formatted.time))
df_date_formatted.show()

+----------+-------+------+------------------+
|      date|   time|userid|         datetime_|
+----------+-------+------+------------------+
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|
|2023-01-01|1:25:00|    u1|2023-01-01 1:25:00|
|2023-01-02|2:05:00|    u1|2023-01-02 2:05:00|
|2023-01-02|2:35:00|    u1|2023-01-02 2:35:00|
|2023-01-02|3:05:00|    u1|2023-01-02 3:05:00|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|
|2023-01-12|1:35:00|    u2|2023-01-12 1:35:00|
+----------+-------+------+------------------+



In [244]:
window=Window().partitionBy("userid").orderBy(F.asc(col("datetime_")))

df_date_formatted=df_date_formatted.withColumn("lag_session",F.lag("datetime_").over(window))
df_date_formatted.show()

+----------+-------+------+------------------+------------------+
|      date|   time|userid|         datetime_|       lag_session|
+----------+-------+------+------------------+------------------+
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|              null|
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|2023-01-01 1:05:00|
|2023-01-01|1:25:00|    u1|2023-01-01 1:25:00|2023-01-01 1:05:00|
|2023-01-02|2:05:00|    u1|2023-01-02 2:05:00|2023-01-01 1:25:00|
|2023-01-02|2:35:00|    u1|2023-01-02 2:35:00|2023-01-02 2:05:00|
|2023-01-02|3:05:00|    u1|2023-01-02 3:05:00|2023-01-02 2:35:00|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|              null|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|2023-01-12 1:05:00|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|2023-01-12 1:05:00|
|2023-01-12|1:35:00|    u2|2023-01-12 1:35:00|2023-01-12 1:05:00|
+----------+-------+------+------------------+------------------+



In [246]:
df_date_formatted=df_date_formatted.withColumn("session_difference",\
                             F.to_timestamp(df_date_formatted.datetime_)\
                             -F.to_timestamp(df_date_formatted.lag_session))
df_date_formatted.show()

+----------+-------+------+------------------+------------------+--------------------+
|      date|   time|userid|         datetime_|       lag_session|  session_difference|
+----------+-------+------+------------------+------------------+--------------------+
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|              null|                null|
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|2023-01-01 1:05:00|INTERVAL '0 00:00...|
|2023-01-01|1:25:00|    u1|2023-01-01 1:25:00|2023-01-01 1:05:00|INTERVAL '0 00:20...|
|2023-01-02|2:05:00|    u1|2023-01-02 2:05:00|2023-01-01 1:25:00|INTERVAL '1 00:40...|
|2023-01-02|2:35:00|    u1|2023-01-02 2:35:00|2023-01-02 2:05:00|INTERVAL '0 00:30...|
|2023-01-02|3:05:00|    u1|2023-01-02 3:05:00|2023-01-02 2:35:00|INTERVAL '0 00:30...|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|              null|                null|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|2023-01-12 1:05:00|INTERVAL '0 00:00...|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:

In [247]:
df_date_formatted.show(truncate=False)

+----------+-------+------+------------------+------------------+-----------------------------------+
|date      |time   |userid|datetime_         |lag_session       |session_difference                 |
+----------+-------+------+------------------+------------------+-----------------------------------+
|2023-01-01|1:05:00|u1    |2023-01-01 1:05:00|null              |null                               |
|2023-01-01|1:05:00|u1    |2023-01-01 1:05:00|2023-01-01 1:05:00|INTERVAL '0 00:00:00' DAY TO SECOND|
|2023-01-01|1:25:00|u1    |2023-01-01 1:25:00|2023-01-01 1:05:00|INTERVAL '0 00:20:00' DAY TO SECOND|
|2023-01-02|2:05:00|u1    |2023-01-02 2:05:00|2023-01-01 1:25:00|INTERVAL '1 00:40:00' DAY TO SECOND|
|2023-01-02|2:35:00|u1    |2023-01-02 2:35:00|2023-01-02 2:05:00|INTERVAL '0 00:30:00' DAY TO SECOND|
|2023-01-02|3:05:00|u1    |2023-01-02 3:05:00|2023-01-02 2:35:00|INTERVAL '0 00:30:00' DAY TO SECOND|
|2023-01-12|1:05:00|u2    |2023-01-12 1:05:00|null              |null             

In [251]:
df_date_formatted=df_date_formatted.withColumn("session_unix_time",
                             F.unix_timestamp(F.to_timestamp("datetime_"))
                            )
df_date_formatted.show()

+----------+-------+------+------------------+------------------+--------------------+-----------------+
|      date|   time|userid|         datetime_|       lag_session|  session_difference|session_unix_time|
+----------+-------+------+------------------+------------------+--------------------+-----------------+
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|              null|                null|             null|
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|2023-01-01 1:05:00|INTERVAL '0 00:00...|                0|
|2023-01-01|1:25:00|    u1|2023-01-01 1:25:00|2023-01-01 1:05:00|INTERVAL '0 00:20...|             1200|
|2023-01-02|2:05:00|    u1|2023-01-02 2:05:00|2023-01-01 1:25:00|INTERVAL '1 00:40...|            88800|
|2023-01-02|2:35:00|    u1|2023-01-02 2:35:00|2023-01-02 2:05:00|INTERVAL '0 00:30...|             1800|
|2023-01-02|3:05:00|    u1|2023-01-02 3:05:00|2023-01-02 2:35:00|INTERVAL '0 00:30...|             1800|
|2023-01-12|1:05:00|    u2|2023-01-12 1:05:00|         

In [255]:
df_date_formatted=df_date_formatted.withColumn("datetime_timestamp",df_date_formatted.datetime_.cast("timestamp"))
df_date_formatted.show()
df_date_formatted.printSchema()

+----------+-------+------+------------------+------------------+--------------------+-----------------+-------------------+
|      date|   time|userid|         datetime_|       lag_session|  session_difference|session_unix_time| datetime_timestamp|
+----------+-------+------+------------------+------------------+--------------------+-----------------+-------------------+
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|              null|                null|             null|2023-01-01 01:05:00|
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|2023-01-01 1:05:00|INTERVAL '0 00:00...|                0|2023-01-01 01:05:00|
|2023-01-01|1:25:00|    u1|2023-01-01 1:25:00|2023-01-01 1:05:00|INTERVAL '0 00:20...|             1200|2023-01-01 01:25:00|
|2023-01-02|2:05:00|    u1|2023-01-02 2:05:00|2023-01-01 1:25:00|INTERVAL '1 00:40...|            88800|2023-01-02 02:05:00|
|2023-01-02|2:35:00|    u1|2023-01-02 2:35:00|2023-01-02 2:05:00|INTERVAL '0 00:30...|             1800|2023-01-02 02:35:00|


In [265]:
#df_date_formatted.withColumn("month",F.expr("extract(month from 'datetime_timestamp')")).show()
df_date_formatted.withColumn("day",F.dayofmonth("datetime_timestamp")).select("day").show()

+-----+
|month|
+-----+
|    1|
|    1|
|    2|
|    2|
|    2|
|   12|
|   12|
|   12|
|    1|
|   12|
+-----+



In [281]:
df_date_formatted=(df_date_formatted
 .withColumn("day",F.date_format("datetime_timestamp","d"))
 .withColumn("month",F.date_format("datetime_timestamp","M"))
 .withColumn("year",F.date_format("datetime_timestamp","y"))
 .withColumn("hour",F.date_format("datetime_timestamp","hh"))
 .withColumn("minute",F.date_format("datetime_timestamp","mm"))
 .withColumn("second",F.date_format("datetime_timestamp","ss")))
#.select("day","month","year","hour","minute","second")
df_date_formatted.show()

+----------+-------+------+------------------+------------------+--------------------+-----------------+-------------------+---+-----+----+----+------+------+
|      date|   time|userid|         datetime_|       lag_session|  session_difference|session_unix_time| datetime_timestamp|day|month|year|hour|minute|second|
+----------+-------+------+------------------+------------------+--------------------+-----------------+-------------------+---+-----+----+----+------+------+
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|              null|                null|             null|2023-01-01 01:05:00|  1|    1|2023|  01|    05|    00|
|2023-01-01|1:05:00|    u1|2023-01-01 1:05:00|2023-01-01 1:05:00|INTERVAL '0 00:00...|                0|2023-01-01 01:05:00|  1|    1|2023|  01|    05|    00|
|2023-01-01|1:25:00|    u1|2023-01-01 1:25:00|2023-01-01 1:05:00|INTERVAL '0 00:20...|             1200|2023-01-01 01:25:00|  1|    1|2023|  01|    25|    00|
|2023-01-02|2:05:00|    u1|2023-01-02 2:05:00|

In [283]:
df_selected=df_date_formatted.select("userid","datetime_timestamp","session_unix_time","year","month","hour","minute","second")
df_selected.show()

+------+-------------------+-----------------+----+-----+----+------+------+
|userid| datetime_timestamp|session_unix_time|year|month|hour|minute|second|
+------+-------------------+-----------------+----+-----+----+------+------+
|    u1|2023-01-01 01:05:00|             null|2023|    1|  01|    05|    00|
|    u1|2023-01-01 01:05:00|                0|2023|    1|  01|    05|    00|
|    u1|2023-01-01 01:25:00|             1200|2023|    1|  01|    25|    00|
|    u1|2023-01-02 02:05:00|            88800|2023|    1|  02|    05|    00|
|    u1|2023-01-02 02:35:00|             1800|2023|    1|  02|    35|    00|
|    u1|2023-01-02 03:05:00|             1800|2023|    1|  03|    05|    00|
|    u2|2023-01-12 01:05:00|             null|2023|    1|  01|    05|    00|
|    u2|2023-01-12 01:05:00|                0|2023|    1|  01|    05|    00|
|    u2|2023-01-12 01:05:00|                0|2023|    1|  01|    05|    00|
|    u2|2023-01-12 01:35:00|             1800|2023|    1|  01|    35|    00|

In [291]:
df_selected=(df_selected.withColumn("timediff_minutes",F.col("session_unix_time")/60)
 .withColumn("timediff_hours",F.round(F.col("session_unix_time")/(60*60))))
df_selected.show()

+------+-------------------+-----------------+----+-----+----+------+------+----------------+--------------+
|userid| datetime_timestamp|session_unix_time|year|month|hour|minute|second|timediff_minutes|timediff_hours|
+------+-------------------+-----------------+----+-----+----+------+------+----------------+--------------+
|    u1|2023-01-01 01:05:00|             null|2023|    1|  01|    05|    00|            null|          null|
|    u1|2023-01-01 01:05:00|                0|2023|    1|  01|    05|    00|             0.0|           0.0|
|    u1|2023-01-01 01:25:00|             1200|2023|    1|  01|    25|    00|            20.0|           0.0|
|    u1|2023-01-02 02:05:00|            88800|2023|    1|  02|    05|    00|          1480.0|          25.0|
|    u1|2023-01-02 02:35:00|             1800|2023|    1|  02|    35|    00|            30.0|           1.0|
|    u1|2023-01-02 03:05:00|             1800|2023|    1|  03|    05|    00|            30.0|           1.0|
|    u2|2023-01-12 

In [219]:
df_date_formatted.dtypes

[('date', 'string'), ('time', 'string'), ('userid', 'string')]

In [203]:
dataframe={"datetime":None,"userid":None}
df_1=[]
for item in val:
    dataframe["datetime"]=item[0]+" "+item[1]
    dataframe["userid"]=item[-1]
    df_1.append(dataframe)
    dataframe={}
df_1

[{'datetime': '2023-01-01 1:05:00', 'userid': 'u1'},
 {'datetime': '2023-01-01 2:05:00', 'userid': 'u1'},
 {'datetime': '2023-01-12 1:05:00', 'userid': 'u2'},
 {'datetime': '2023-01-01 1:05:00', 'userid': 'u1'},
 {'datetime': '2023-01-12 1:35:00', 'userid': 'u2'}]

+----------+-------+---+---+---+---+---+---+
|        _1|     _2| _3| _4| _5| _6| _7| _8|
+----------+-------+---+---+---+---+---+---+
|2023-01-01|1:05:00|   |   |   |   |   | u1|
|2023-01-01|1:25:00|   |   |   |   |   | u1|
|2023-01-02|2:05:00|   |   |   |   |   | u1|
|2023-01-02|2:35:00|   |   |   |   |   | u1|
|2023-01-02|3:05:00|   |   |   |   |   | u1|
|2023-01-12|1:05:00|   |   |   |   |   | u2|
|2023-01-12|1:05:00|   |   |   |   |   | u2|
|2023-01-12|1:05:00|   |   |   |   |   | u2|
|2023-01-01|1:05:00|   |   |   |   |   | u1|
|2023-01-12|1:35:00|   |   |   |   |   | u2|
+----------+-------+---+---+---+---+---+---+



In [121]:
schema ="""
timestamp timestamp,
userid string
"""
arr=[]
with open(filepath,"r") as f:
    arr.append(f.readlines())


In [131]:
array_dataframe=arr[0][7:]
array_dataframe

['2023-01-01 1:05:00  u1\n',
 '2023-01-01 2:05:00  u1\n',
 '2023-01-12 1:05:00  u2\n',
 '2023-01-01 1:05:00  u1\n',
 '2023-01-12 1:35:00  u2']

In [136]:
array_dataframe[0].split(" ")

['2023-01-01', '1:05:00', '', 'u1\n']